In [2]:
import polars as pl

In [3]:
college = pl.read_csv("data/electoral_college.csv")

In [4]:
college

Full_State,Abb_State,Electoral_College_Votes,banzhaf_power_index
str,str,i64,f64
"""Alabama""","""AL""",9,1.6
"""Alaska""","""AK""",3,0.7
"""Arizona""","""AZ""",11,2.0
"""Arkansas""","""AR""",6,1.1
"""California""","""CA""",55,11.4
…,…,…,…
"""Virginia""","""VA""",13,2.4
"""Washington""","""WA""",12,2.2
"""West Virginia""","""WV""",5,0.9


In [5]:
state_groups = pl.read_csv("data/state_groups2.csv")
state_groups

state,group
str,i64
"""Alabama""",1
"""Mississippi""",1
"""Louisiana""",1
"""Oklahoma""",1
"""Alaska""",1
…,…
"""Utah""",16
"""Virginia""",17
"""Minnesota""",18


In [6]:
joined = college.join(state_groups, left_on="Full_State", right_on="state", how="inner")
joined

Full_State,Abb_State,Electoral_College_Votes,banzhaf_power_index,group
str,str,i64,f64,i64
"""Alabama""","""AL""",9,1.6,1
"""Mississippi""","""MS""",6,1.1,1
"""Louisiana""","""LA""",8,1.5,1
"""Oklahoma""","""OK""",7,1.3,1
"""Alaska""","""AK""",3,0.7,1
…,…,…,…,…
"""Utah""","""UT""",6,1.1,16
"""Virginia""","""VA""",13,2.4,17
"""Minnesota""","""MN""",10,1.8,18


In [ ]:
joined.group_by("group").agg(
    pl.sum("Electoral_College_Votes").alias("total_votes"),
    pl.sum("banzhaf_power_index").alias("total_banzhaf_power"),
    pl.col("Full_State").str.concat(delimiter=",").alias("states"),
).write_csv("data/electoral_college_banzhaf2.csv")

In [3]:
all_data = pl.read_csv("data/electoral_college_banzhaf.csv")
all_data

group,total_votes,total_banzhaf_power,states,grouped_index
i64,i64,f64,str,f64
1,85,15.9,"""Alabama,Mississippi,Louisiana,…",0.104511
5,20,3.7,"""Pennsylvania""",0.050962
16,6,1.1,"""Utah""",0.01523
20,4,0.7,"""New Hampshire""",0.010219
2,186,35.8,"""California,Connecticut,Delawar…",0.902917
…,…,…,…,…
4,11,2.0,"""Arizona""",0.027973
7,29,5.4,"""Florida""",0.073763
18,10,1.8,"""Minnesota""",0.025414


In [4]:
all_data = all_data.with_columns(
    total_banzhaf_power=pl.col("total_banzhaf_power") / 100,
    grouped_index=pl.col("grouped_index") / pl.sum("grouped_index"),
).select(["states", "total_banzhaf_power", "grouped_index"])
all_data

states,total_banzhaf_power,grouped_index
str,f64,f64
"""Alabama,Mississippi,Louisiana,…",0.159,0.063116
"""Pennsylvania""",0.037,0.030777
"""Utah""",0.011,0.009198
"""New Hampshire""",0.007,0.006171
"""California,Connecticut,Delawar…",0.358,0.54529
…,…,…
"""Arizona""",0.02,0.016894
"""Florida""",0.054,0.044547
"""Minnesota""",0.018,0.015348


In [5]:
all_data.write_csv("data/electoral_college_banzhaf_all_data.csv")